# Preprocesamiento imágenes cultivos de arroz

## Índice
0. [Objetivo y alcance](#objective)
1. [Análisis inicial](#analysis)
  1. [Análisis imagen 1](#analysis1)
  2. [Análisis imagen 2](#analysis2)
  3. [Análisis imagen 3](#analysis3)
  4. [Análisis imagen 4](#analysis4)
2. [Medidas de eficiencia](#measures)
3. [Algoritmo transformación](#conversion)
4. [Resultados](#results)  
  1. [Resultados imagen 1](#res1)
  2. [Resultados imagen 2](#res2)
  3. [Resultados imagen 3](#res3)
  4. [Resultados imagen 4](#res4)
7. [Referencias](#references)


## <a name="objective"></a>0. Objetivo y alcance
El objetivo de este notebook es presentar las técnicas utilizadas para el preprocesamiento de imágenes de cultivo de arroz.

Las técnicas utilizadas están orientadas a identificar el área de masa vegetal, para ello utilizarán técnicas de mejora de contraste, normalización del brillo, transformación del espacio cromático, erosión y dilatación matricial.


Ejemplos de imágenes de entrada:
<table>
    <tr>
        <td><img src="images/campo01.png" alt="Ejemplo imagen 1" style="width:200px;height:200px"/></td>
        <td><img src="images/campo02.png" alt="Ejemplo imagen 2" style="width:200px;height:200px"/></td>
        <td><img src="images/campo03.jpg" alt="Ejemplo imagen 3" style="width:200px;height:200px"/></td>
        <td><img src="images/campo04.jpeg" alt="Ejemplo imagen 4" style="width:200px;height:200px"/></td>
    </tr>
</table>

## <a name="analysis"></a>1. Análisis inicial

El análisis inicial contiene métodos para la visualización del histograma utilizando el modelo de color RGB. Además, contiene el método para la ecualización de dicho histograma.

In [ ]:
import numpy as np
import cv2 as cv
import math as mt
import matplotlib.pyplot as plt


def get_image(image_path):
    """Get image object (np array) given a path"""
    return cv.imread(image_path)


def show_histogram(image, title='histogram'):
    """Show Histogram for BGR image"""
    color = ('b','g','r')
    for i,col in enumerate(color):
        hist = cv.calcHist([image],[i],None,[256],[0,256])
        plt.plot(hist,color = col)
        plt.xlim([0,256])

    plt.title(title)
    plt.show()

    
def show_histogram_channel(image, channel):
    """Show Histogram specifying the channel (0: blue,1:green,2:red)"""
    color = ('b','g','r')
    
    hist = cv.calcHist([image],[channel],None,[256],[0,256])
    plt.plot(hist,color = color[channel])
    plt.xlim([0,256])
    plt.show()

    
def get_image_histogram_equalization(image):
    """
    Calculate histogram equalization 
    - Returns: image with equalized histogram
    """
    
    # Split BGR channels
    blue, green, red = cv.split(image)
    
    # Equalize histogram for each channel independtly
    blue_equalized = cv.equalizeHist(blue)
    green_equalized = cv.equalizeHist(green)
    red_equalized = cv.equalizeHist(red)

    # Merge BRG channels
    image_full_equalized = cv.merge((blue_equalized,green_equalized,red_equalized))
    
    return image_full_equalized


def visualize(image1, image2):
    """Show 2 images in a 2 grid"""
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(30,30))
    ax1.imshow(cv.cvtColor(image1, cv.COLOR_BGR2RGB))
    ax2.imshow(cv.cvtColor(image2, cv.COLOR_BGR2RGB))

### <a name="analysis1"></a>Análisis imagen 1

In [ ]:
image_path_1 = './images/campo01.png'
image_1 = get_image(image_path_1)
image_equalized_1 = get_image_histogram_equalization(image_1)
visualize(image_1, image_equalized_1)

In [ ]:
show_histogram(image_1, 'image_original histogram')
show_histogram(image_equalized_1, 'image_equalized_histogram')

### <a name="analysis2"></a>Análisis imagen 2

In [ ]:
image_path_2 = './images/campo02.png'
image_2 = get_image(image_path_2)
image_equalized_2 = get_image_histogram_equalization(image_2)
visualize(image_2, image_equalized_2)

In [ ]:
show_histogram(image_2, 'image_original histogram')
show_histogram(image_equalized_2, 'image_equalized_histogram')

### <a name="analysis3"></a>Análisis imagen 3

In [ ]:
image_path_3 = './images/campo03.jpg'
image_3 = get_image(image_path_3)
image_equalized_3 = get_image_histogram_equalization(image_3)
visualize(image_3, image_equalized_3)

In [ ]:
show_histogram(image_3, 'image_original histogram')
show_histogram(image_equalized_3, 'image_equalized_histogram')

### <a name="analysis4"></a>Análisis imagen 4

In [ ]:
image_path_4 = './images/campo04.jpeg'
image_4 = get_image(image_path_4)
image_equalized_4 = get_image_histogram_equalization(image_4)
visualize(image_4, image_equalized_4)

In [ ]:
show_histogram(image_4, 'image_original histogram')
show_histogram(image_equalized_4, 'image_equalized_histogram')

## <a name="measures"></a>2. Medidas de eficiencia, contando objetos

Los siguientes métodos utilizan el algoritmo de Canny para tratar de contabilizar el número de plantas de arroz para poder contrastar la eficiencia del algoritmo implementado como solución a los problemas de ruido (Algoritmo BN).

El algoritmo de Canny es un popular método de detección de bordes, desarrollado por John F. Canny en 1986

In [ ]:
def count_plants(image, image_transformed, title1="Imagen original", title2="Imagen transformada", EstimacionManual = 0, description="Resultados"):
    """
        Get edges of objects in the images and count them
    """    

    ### 1. Imagen original
    # Convertimos a escala de grises
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
    # Aplicar suavizado Gaussiano
    gauss = cv2.GaussianBlur(gris, (5,5), 0)
  
    # Detectamos los bordes con Canny
    canny = cv.Canny(gauss, 50, 150)
  
    # Buscamos los contornos
    (contornos_image,_) = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image, contornos_image,-1,(0,0,255), 2)
    
    
    ### 2. Imagen transformada
    ## Imagen transformada
    gaussTrans = cv2.GaussianBlur(image_transformed, (5,5), 0)

    cannyTrnas = cv2.Canny(gaussTrans, 50, 150)
 
    # Buscamos los contornos
    (contornos_image_transformed,_) = cv2.findContours(cannyTrnas.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)       

    cv2.drawContours(image_transformed,contornos_image_transformed,-1,(0,0,255), 2)
    
    
    ### Cálculos
    mejora = 0
    if (len(contornos_image_transformed) > 0):
        mejora = round(((len(contornos_image_transformed) -len(contornos_image)) / len(contornos_image_transformed)) * 100,2)

    print(" ")
    print(description)
    print("----------------------------------------------- ")
    print("Se encontraron", format(len(contornos_image)), " objetos en la imagen original y ", format(len(contornos_image_transformed)), " en la transformada")
    diferencia_original = abs(EstimacionManual - len(contornos_image))
    diferencia_transformada = abs(EstimacionManual - len(contornos_image))


    f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey = False, figsize=(50,50))
    ax1.set_title(title1)
    ax1.imshow(image, extent=(0, 128, 128, 0), interpolation='nearest', cmap="gray")     
    ax2.set_title(title2)
    ax2.imshow(image_transformed, extent=(0, 128, 128, 0), interpolation='nearest', cmap="gray") 
    
    #return contornos


## <a name="conversion"></a> 3. Transformaciones 


#### Algoritmo BN

El resultado del algoritmo será una imagen en blanco y negro, donde:

    - Blanco: Representa área ocupada por materia vegetal.
    - Negro: Todo lo no considerado como vegetal.

Entrada / Salida

    - Entrada: una imagen en color (formato .png .jpeg .jpg)
    - Salida: array numpy de una imagen en blanco y negro.

Funcionamiento:

Una característica típica de las plantas herbáceas es su color verde, resultante de la absorción de los fotones rojos y azules y la reflexión de los verdes.

El objetivo del algoritmo es identificar aquellas zonas donde las frecuencias de luz ROJAS y AZULES han sido absorvidas por la materia.

Como paso previo a la conversión realizaremos una equalización de la imagen, con ella normalizaremos el brillo e incrementaremos su contraste.

Una vez ecualizada, el algoritmo recorrerá los píxeles de la foto convirtiéndolos a su representación R,G,B.

Finalmente aplicaremos una Erosión y una posterior Dilatación para eliminar los puntos blancos sueltos.

Para identificar la masa vegetal comparará los valores R y B (rojo y azul) con los valores globales medios descontando su desviación típica.

Si el valor en ambas bandas: 

    - es menor que el valor medio menos su desviación típica: se cambia a Blanco (es masa vegetal)    
    - no es menor: se cambia a Negro (no es masa vegetal)
   
#### 


In [ ]:
import cv2

def ConvierteBN_Erosion_Dilatacion_CamposArroz(imageObj):

    #Descomponemos la imagen en sus canales RGB
    blue, green, red = cv2.split(imageObj)
    
    # Cálculo de medias y varianzas para establecer el umbral.
    media_azul = np.mean(blue)
    media_rojo = np.mean(red)
    media_verde = np.mean(green)
    desviacion_azul = mt.sqrt(np.var(blue))
    desviacion_rojo = mt.sqrt(np.var(red))
    desviacion_verde = mt.sqrt(np.var(green))

    #Cáculo de los umbrales
    UMB_r = media_rojo - desviacion_rojo
    UMB_b = media_azul - desviacion_azul
    UMB_g = media_verde - desviacion_verde
    row, col, pixel = imageObj.shape

    # Construimos los canales para su visualización	
    zeros = np.zeros(blue.shape, np.uint8)	
    blueBGR = cv2.merge((blue,zeros,zeros))
    greenBGR = cv2.merge((zeros,green,zeros))
    redBGR = cv2.merge((zeros,zeros,red))

    # or ( (media_verde - desviacion_verde) < green[x,y] < (media_verde + desviacion_verde))    

    for x in range(0, row, 1):
        for y in range(0, col, 1):
            if ( ( (blue[x,y] < UMB_b)  and (red[x,y] < UMB_r) ) ):
                blue[x,y] = 255
                red[x,y]= 255
                green[x,y]=255
            else:
                
                blue[x,y] = 0
                red[x,y]=  0
                green[x,y]= 0
    
    # Volvemos a unir los canales para la salida.
    unionBGR = cv2.merge((blue,green,red))
    
    # Aplicamos la erosión y dilatación para eliminar puntos blanco sueltos
    kernel_erosion = np.ones((3,3), np.uint8)
    img_erosion = cv2.erode(unionBGR, kernel_erosion, iterations=1)
    kernel_dilatacion = np.ones((5,5), np.uint8)
    unionBGR = cv2.dilate(img_erosion, kernel_dilatacion, iterations=2)

    # Mostramos los canales y la imagen resultado
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex=False, sharey = False, figsize=(150,150))

    ax1.set_title("Original azul")
    ax2.set_title("Original verde")
    ax3.set_title("Original rojo")
    ax4.set_title("Unión Transformada")
    
    ax1.imshow(blueBGR, extent=(0, 128, 128, 0), interpolation='nearest', cmap="gray")
    ax2.imshow(greenBGR, extent=(0, 128, 128, 0), interpolation='nearest', cmap="gray")
    ax3.imshow(redBGR, extent=(0, 128, 128, 0), interpolation='nearest', cmap="gray")
    ax4.imshow(unionBGR, extent=(0, 128, 128, 0), interpolation='nearest', cmap="gray")          

    return unionBGR

# <a name="results"></a>Resultados

### <a name="res1"></a>Resultados Imagen 1

In [ ]:
"""
    >> Resultados Pre-Procesamiento Imagen 1
    0. Captura Imagen original
    1. Algoritmo BN (con erosión + dilatación)
    3. Visualización imagen original y resultado
"""
image_path_1 = './images/campo01.png'
image_1 = get_image(image_path_1)

result_image_bn_1 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_1)

In [ ]:
"""
    Imagen original vs Imagen original + Algoritmo BN
"""
visualize(image_1, result_image_bn_1)   

In [ ]:
"""
    >> Resultados Pre-Procesamiento Imagen 1
    0. Captura Imagen original
    1. Ecualización histograma imagen
    2. Algoritmo BN (con erosión + dilatación)
    3. Visualización imagen original y resultado
"""
image_equalized_1 = get_image_histogram_equalization(image_1)
result_image_equalized_bn_1 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_equalized_1)

In [ ]:
"""
    Imagen original vs Imagen original + Ecualización + Algoritmo BN
"""
visualize(image_1, result_image_equalized_bn_1)

In [ ]:
"""
    Comprobación resultados con algoritmo canny
"""
image_path_1 = './images/campo01.png'
image_1 = get_image(image_path_1)
result_image_bn_1 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_1)
count_plants(image_1, result_image_bn_1, description= "Algoritmo BN + Erosion&Dilatacion")

image_path_1 = './images/campo01.png'
image_1 = get_image(image_path_1)
image_equalized_1 = get_image_histogram_equalization(image_1)
result_image_equalized_bn_1 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_equalized_1)
count_plants(image_1, result_image_equalized_bn_1, description= "Ecualización + Algoritmo BN + Erosion&Dilatacion")

### <a name="res2"></a>Resultados Imagen 2

In [ ]:

"""
    >> Resultados Pre-Procesamiento Imagen 2
    0. Captura Imagen original
    1. Algoritmo BN (con erosión + dilatación)
    2. Visualización imagen original y resultado
"""
image_path_2 = './images/campo02.png'
image_2 = get_image(image_path_2)

result_image_bn_2 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_2)

In [ ]:
"""
    Imagen original vs Imagen original + Algoritmo BN
"""
visualize(image_2, result_image_bn_2)  

In [ ]:

"""
    >> Resultados Pre-Procesamiento Imagen 2
    0. Captura Imagen original
    1. Ecualización histograma imagen
    2. Algoritmo BN (con erosión + dilatación)
    3. Visualización imagen original y resultado
"""
image_equalized_2 = get_image_histogram_equalization(image_2)
result_image_equalized_bn_2 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_equalized_2)

In [ ]:
"""
    Imagen original vs Imagen original + Ecualización + Algoritmo BN
"""
visualize(image_2, result_image_equalized_bn_2)

In [ ]:
"""
    Comprobación resultados con algoritmo canny
"""
image_path_2 = './images/campo02.png'
image_2 = get_image(image_path_2)
result_image_bn_2 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_2)
count_plants(image_2, result_image_bn_2, description= "Algoritmo BN + Erosion&Dilatacion")

image_path_2 = './images/campo02.png'
image_2 = get_image(image_path_2)
image_equalized_2 = get_image_histogram_equalization(image_2)
result_image_equalized_bn_2 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_equalized_2)
count_plants(image_2, result_image_equalized_bn_2, description= "Ecualización + Algoritmo BN + Erosion&Dilatacion")

### <a name="res3"></a>Resultados Imagen 3

In [ ]:
"""
    >> Resultados Pre-Procesamiento Imagen 3
    0. Captura Imagen original
    1. Algoritmo BN (con erosión + dilatación)
    2. Visualización imagen original y resultado
"""
image_path_3 = './images/campo03.jpg'
image_3 = get_image(image_path_3)

result_image_bn_3 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_3)

In [ ]:
"""
    Imagen original vs Imagen original + Algoritmo BN
"""
visualize(image_3, result_image_bn_3)   

In [ ]:
"""
    Imagen original vs Imagen original + Algoritmo BN
"""
visualize(image_3, result_image_bn_3)   


"""
    >> Resultados Pre-Procesamiento Imagen 3
    0. Captura Imagen original
    1. Ecualización histograma imagen
    2. Algoritmo BN (con erosión + dilatación)
    3. Visualización imagen original y resultado
"""
image_equalized_3 = get_image_histogram_equalization(image_3)
result_image_equalized_bn_3 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_equalized_3)

In [ ]:
"""
    Imagen original vs Imagen original + Ecualización + Algoritmo BN
"""
visualize(image_3, result_image_equalized_bn_3)

In [ ]:
"""
    Comprobación resultados con algoritmo canny
"""
image_path_3 = './images/campo03.jpg'
image_3 = get_image(image_path_3)
result_image_bn_3 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_3)
count_plants(image_3, result_image_bn_3, description= "Algoritmo BN + Erosion&Dilatacion")

image_path_3 = './images/campo03.jpg'
image_3 = get_image(image_path_3)
image_equalized_3 = get_image_histogram_equalization(image_3)
result_image_equalized_bn_3 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_equalized_3)
count_plants(image_3, result_image_equalized_bn_3, description= "Ecualización + Algoritmo BN + Erosion&Dilatacion")

### <a name="res4"></a>Resultados Imagen 4

In [ ]:
"""
    >> Resultados Pre-Procesamiento Imagen 4
    0. Captura Imagen original
    1. Algoritmo BN (con erosión + dilatación)
    2. Visualización imagen original y resultado
"""
image_path_4 = './images/campo04.jpeg'
image_4 = get_image(image_path_4)

result_image_bn_4 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_4)

In [ ]:
"""
    Imagen original vs Imagen original + Algoritmo BN
"""
visualize(image_4, result_image_bn_4)   

In [ ]:
"""
    >> Resultados Pre-Procesamiento Imagen 4
    0. Captura Imagen original
    1. Ecualización histograma imagen
    2. Algoritmo BN (con erosión + dilatación)
    3. Visualización imagen original y resultado
"""
image_equalized_4 = get_image_histogram_equalization(image_4)
result_image_equalized_bn_4 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_equalized_4)

In [ ]:
"""
    Imagen original vs Imagen original + Ecualización + Algoritmo BN
"""
visualize(image_4, result_image_equalized_bn_4)

In [ ]:
"""
    Comprobación resultados con algoritmo canny
"""
image_path_4 = './images/campo04.jpeg'
image_4 = get_image(image_path_4)
result_image_bn_4 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_4)
count_plants(image_4, result_image_bn_4, description= "Algoritmo BN + Erosion&Dilatacion")

image_path_4 = './images/campo04.jpeg'
image_4 = get_image(image_path_4)
image_equalized_4 = get_image_histogram_equalization(image_4)
result_image_equalized_bn_4 = ConvierteBN_Erosion_Dilatacion_CamposArroz(image_equalized_4)
count_plants(image_4, result_image_equalized_bn_4, description= "Ecualización + Algoritmo BN + Erosion&Dilatacion")

## <a name="references"></a> Referencias


- Color Systems
http://color.lukas-stratmann.com/color-systems/rgb.html

- Get histogram for image
https://docs.opencv.org/4.x/d1/db7/tutorial_py_histogram_begins.html

- Histogram equalization
https://docs.opencv.org/3.4/d4/d1b/tutorial_histogram_equalization.html

- Detector de bordes Canny cómo contar objetos con OpenCV y Python
https://programarfacil.com/blog/vision-artificial/detector-de-bordes-canny-opencv/#Algoritmo_para_contar_objetos_con_OpenCV
